# Doing stats on the tagged data from the dashboard

This script written by **Shah Nafis Rafique** will attempt to find out any meaningful information from the tagged data that is made from the dashboard

Import the imports

In [10]:
import pandas as pd
import jsonpickle as jsonpickle
import numpy as np
import json

In [11]:
JSON_FILE_TO_IMPORT = "../../../../../../tagging_data_personal.json" #"test.json"

Import the `tagging_data_personal.json` file. We have to use `jsonpickle` since thats what Matt uses

In [12]:
with open(JSON_FILE_TO_IMPORT, 'r') as f:
    data = jsonpickle.decode(f.read())
    print(f"Keys of the JSON are {list(data.keys())}")

Keys of the JSON are ['catalog_path', 'current_image', 'debug', 'finished_tagged_queue', 'last_backup_timestamp', 'max_skipped_queue', 'pending_images_queue', 'py/object', 'scope_path', 'small_path']


6056461.jpg
jpgs/P26057016.jpg
jpgs/P26050223.jpg
jpgs/P26056310.jpg
jpgs/P26047728.jpg
jpgs/C26048098.jpg
jpgs/P26047325.jpg
jpgs/P26058912.jpg
jpgs/P26058556.jpg
jpgs/P26056899.jpg
jpgs/P26050232.jpg
jpgs/C26049134.jpg
jpgs/C26048077.jpg
jpgs/P26056474.jpg
jpgs/P26048734.jpg
jpgs/P26047295.jpg
jpgs/P26056718.jpg
jpgs/P26056615.jpg
jpgs/P26047718.jpg
jpgs/C26047975.jpg
jpgs/S26047353.jpg
jpgs/P26047470.jpg
jpgs/P26049692.jpg
jpgs/P26048276.jpg
jpgs/P26054287.jpg
jpgs/P26059416.jpg
jpgs/P26047918.jpg
jpgs/P26050422.jpg
jpgs/P26056998.jpg
jpgs/P26057367.jpg
jpgs/P26052026.jpg
jpgs/P26055782.jpg
jpgs/C26047250.jpg
jpgs/P26050969.jpg
jpgs/P26059823.jpg
jpgs/P26058469.jpg
jpgs/P26060188.jpg
jpgs/P26047893.jpg
jpgs/P26057629.jpg
jpgs/P26052840.jpg
jpgs/C26047416.jpg
jpgs/P26047945.jpg
jpgs/P26048370.jpg
jpgs/P26050289.jpg
jpgs/P26060067.jpg
jpgs/P26055499.jpg
jpgs/C26049173.jpg
jpgs/P26047244.jpg
jpgs/C26048288.jpg
jpgs/P26056585.jpg
jpgs/P26047446.jpg
jpgs/P26058124.jpg
jpgs/P26051006.jpg
